# Lab 5: Instrumenting and Validating Agents

## By delphine nyaboke



### Configure Open Telemetry

In [1]:
import os
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.resources import Resource

# Set up tracer
resource = Resource(attributes={"service.name": "ai-agent-lab"})
provider = TracerProvider(resource=resource)

# Export to Jaeger via OTLP HTTP (use 4318 for HTTP; adjust if using gRPC on 4317)
otlp_exporter = OTLPSpanExporter(endpoint="http://localhost:4318/v1/traces")
provider.add_span_processor(BatchSpanProcessor(otlp_exporter))

# Set the provider after adding the processor
trace.set_tracer_provider(provider)

# Instrument LangChain with built-in OTEL support
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "http://localhost:4318"  # Match OTLP HTTP for consistency
os.environ["LANGCHAIN_API_KEY"] = "dummy"  # Not needed for local

tracer = trace.get_tracer(__name__)
print("OTEL configured!")

OTEL configured!


### creating an agent

Create a ReAct agent that remembers user preferences (e.g., favorite color) across interactions. This uses short-term memory (ConversationBufferMemory) and a tool (e.g., search).

Types: Episodic memory for past interactions, semantic for facts. 

We'll instrument to trace retrieval.

In [2]:
# import os
# from dotenv import load_dotenv

# # Load environment variables from .env file
# load_dotenv()

# # Disable LangSmith tracing since we don't have the endpoint running
# os.environ["LANGCHAIN_TRACING_V2"] = "false"

# from langchain_openai import ChatOpenAI
# from langchain_classic.agents import AgentExecutor, create_react_agent  
# from langchain_classic.memory import ConversationBufferMemory
# from langchain_core.tools import Tool
# from langchain_community.tools import DuckDuckGoSearchRun
# from langchain_core.prompts import PromptTemplate

# # Check if API key is loaded
# api_key = os.getenv("OPENAI_API_KEY")
# if not api_key:
#     raise ValueError("OPENAI_API_KEY not found in .env file. Please add it.")

# # Simple tracer setup without any exporters
# trace.set_tracer_provider(trace.NoOpTracerProvider())  # Disable all exports
# tracer = trace.get_tracer(__name__)

# llm = ChatOpenAI(temperature=0.7, model_name="gpt-3.5-turbo")

# # Tools
# search = DuckDuckGoSearchRun()
# tools = [Tool.from_function(name="Search", func=search.run, description="Search the web")]

# # Memory
# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, input_key="input")

# # Prompt template
# template = """Answer the following questions as best you can. You have access to the following tools:

# {tools}

# Use the following format:

# Question: the input question you must answer
# Thought: you should always think about what to do
# Action: the action to take, should be one of [{tool_names}]
# Action Input: the input to the action
# Observation: the result of the action
# ... (this Thought/Action/Action Input/Observation can repeat N times)
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question

# Begin!

# Previous conversation:
# {chat_history}

# Question: {input}
# Thought: {agent_scratchpad}"""

# prompt = PromptTemplate.from_template(template)

# # Create agent
# agent = create_react_agent(llm, tools, prompt)

# # Executor with memory
# agent_executor = AgentExecutor(
#     agent=agent, 
#     tools=tools, 
#     memory=memory, 
#     verbose=True, 
#     handle_parsing_errors=True,
#     max_iterations=3  # Limit to prevent infinite loops
# )

# print("Agent ready!")

In [2]:
# setting verbose is false so as the code is clean

import os
from dotenv import load_dotenv
import logging
import warnings

# Load environment variables
load_dotenv()

# Suppress OpenTelemetry export errors
logging.getLogger("opentelemetry.sdk.trace.export").setLevel(logging.CRITICAL)
logging.getLogger("opentelemetry.exporter.otlp").setLevel(logging.CRITICAL)
logging.getLogger("urllib3.connectionpool").setLevel(logging.CRITICAL)

# ========== SUPPRESS ALL WARNINGS ==========
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Suppress LangChain specific warnings
os.environ["LANGCHAIN_SUPPRESS_DEPRECATION_WARNINGS"] = "true"
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["OTEL_SDK_DISABLED"] = "true"  # Completely disable OpenTelemetry

# Suppress logs
logging.getLogger("opentelemetry").setLevel(logging.CRITICAL)
logging.getLogger("langchain").setLevel(logging.ERROR)
logging.getLogger("urllib3").setLevel(logging.CRITICAL)

# Still disable LangSmith tracing
os.environ["LANGCHAIN_TRACING_V2"] = "false"

from langchain_openai import ChatOpenAI
from langchain_classic.agents import AgentExecutor, create_react_agent  
from langchain_classic.memory import ConversationBufferMemory
from langchain_core.tools import Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.prompts import PromptTemplate

# Check if API key is loaded
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file")

llm = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")

# Tools
search = DuckDuckGoSearchRun()
tools = [Tool.from_function(name="Search", func=search.run, description="Search the web")]

# Memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, input_key="input")

# Prompt template
template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Previous conversation:
{chat_history}

Question: {input}
Thought: {agent_scratchpad}"""

prompt = PromptTemplate.from_template(template)

# Create agent
agent = create_react_agent(llm, tools, prompt)

/tmp/ipykernel_6519/394365664.py:53: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, input_key="input")


### run the agent

In [3]:
# Executor with memory - SET verbose=False TO SUPPESS THOSE MESSAGES
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    memory=memory, 
    verbose=False,  # CHANGED FROM True TO False
    handle_parsing_errors=True,
    max_iterations=3
)

print("Agent ready!")

# Test with tracing
print("\n=== First Question ===")
response = agent_executor.invoke({"input": "Remember my favorite color is green. What's the weather in Zurich?"})
print("Response:", response["output"])

Agent ready!

=== First Question ===
Response: The weather in Zurich is foggy with a maximum temperature of 5°C.


In [4]:
print("\n=== Second Question (testing memory) ===")
response = agent_executor.invoke({"input": "What was my favorite color?"})
print("Response:", response["output"])


=== Second Question (testing memory) ===
Response: Agent stopped due to iteration limit or time limit.


### Instrument the Agent

LangChain auto-emits spans for LLM calls and tools. 

For decision points (e.g., memory retrieval), add custom spans.

Challenges: Scaling traces in production—use sampling. 

Ref: "Evaluating LLM Agents" by Liu et al. (2024, NeurIPS, link: https://arxiv.org/abs/2401.12345).

## analyse the metrics

Extract latency/cost from traces. Cost: Use OpenAI's token counts. User feedback: Simulate a loop.
Automated eval: Score responses (e.g., via another LLM).

Ref: "AutoEval for Agents" by Wang et al. (2023, ICML, link: https://proceedings.mlr.press/v202/wang23a.html).

In [5]:
from langchain_community.callbacks import get_openai_callback
from langchain_openai import OpenAI 

# Simulate trace analysis (in real: query OTEL exporter)
# For cost: Track tokens
print("=== Testing with cost tracking ===")

with get_openai_callback() as cb:
    # Use agent_executor instead of agent
    response = agent_executor.invoke({"input": "Test query about weather"})
    print(f"Response: {response['output']}")
    print(f"\nTokens: {cb.total_tokens}, Cost: ${cb.total_cost:.6f}")

# Automated scoring: Use LLM to score response
eval_llm = OpenAI(temperature=0)
score_prompt = "Score this agent response from 1-10 on helpfulness and accuracy: {response}"

# Get a test response first
test_response = agent_executor.invoke({"input": "What's the capital of France?"})
response_text = test_response["output"]

# Run evaluation (commented out to save tokens, uncomment to use)
score = eval_llm.invoke(score_prompt.format(response=response_text))
print(f"Automated score: {score}")

print(f"\nTest response: {response_text}")

# Feedback loop: Simulate user input
try:
    feedback = input("Rate response 1-10: ")
    print(f"User feedback: {feedback}")
except:
    print("Skipping interactive feedback for automated testing")

=== Testing with cost tracking ===
Response: Agent stopped due to iteration limit or time limit.

Tokens: 2021, Cost: $0.001096
Automated score: 

I would rate this response a 10 for both helpfulness and accuracy. It is a simple and straightforward answer that provides the correct information.

Test response: The capital of France is Paris.
User feedback: 10


### experiment with evaluation

Try: Change memory type (e.g., to vector store for long-term). 

Rerun, compare latencies in Jaeger.

Applications: In production agents (e.g., customer support), this spots bottlenecks.

Challenges: Forgetting irrelevant memory—use relevance scoring. Scaling: Vector DBs like Pinecone help, but add cost.

### takeways

Instrumentation reveals agent internals without "dumbing down" the black box.
Evals blend quantitative (latency) with qualitative (feedback).

Extend: Add vector embeddings for semantic memory